In [14]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-03-24 21:48:03.264543: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-24 21:48:03.492302: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 21:48:03.492330: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 21:48:03.492353: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 21:48:03.500522: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-03-24 21:48:06.048404: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 21:48:06.095800: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 21:48:06.096056: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [8]:
import os 
sorted(os.listdir("BIRDS1_split/train_165"))[35]

'blue_gray_gnatcatcher'

In [10]:
import os 
sorted(os.listdir("BIRDS1_split/train"))[35]

'black_vulture'

In [2]:
train_data_path = "BIRDS1_split/train"
test_data_path = "BIRDS1_split/test"
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
bird_classes = sorted(os.listdir(train_data_path))

In [27]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)


Found 25744 files belonging to 167 classes.
Using 18021 files for training.
Found 25744 files belonging to 167 classes.
Using 7723 files for validation.
Found 11545 files belonging to 167 classes.


In [28]:
preprocess_input = tf.keras.applications.mobilenet_v3.preprocess_input
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_dataset = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [5]:
base_model = tf.keras.applications.MobileNetV3Large(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)
base_model.trainable = False

for layer in base_model.layers:
    print(layer.name)

input_1
rescaling
Conv
Conv/BatchNorm
tf.math.add
re_lu
tf.math.multiply
multiply
expanded_conv/depthwise
expanded_conv/depthwise/BatchNorm
re_lu_1
expanded_conv/project
expanded_conv/project/BatchNorm
expanded_conv/Add
expanded_conv_1/expand
expanded_conv_1/expand/BatchNorm
re_lu_2
expanded_conv_1/depthwise/pad
expanded_conv_1/depthwise
expanded_conv_1/depthwise/BatchNorm
re_lu_3
expanded_conv_1/project
expanded_conv_1/project/BatchNorm
expanded_conv_2/expand
expanded_conv_2/expand/BatchNorm
re_lu_4
expanded_conv_2/depthwise
expanded_conv_2/depthwise/BatchNorm
re_lu_5
expanded_conv_2/project
expanded_conv_2/project/BatchNorm
expanded_conv_2/Add
expanded_conv_3/expand
expanded_conv_3/expand/BatchNorm
re_lu_6
expanded_conv_3/depthwise/pad
expanded_conv_3/depthwise
expanded_conv_3/depthwise/BatchNorm
re_lu_7
expanded_conv_3/squeeze_excite/AvgPool
expanded_conv_3/squeeze_excite/Conv
expanded_conv_3/squeeze_excite/Relu
expanded_conv_3/squeeze_excite/Conv_1
tf.math.add_1
re_lu_8
tf.math.mul

In [6]:
from keras.src import regularizers

base_model = tf.keras.applications.MobileNetV3Large(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)
inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name="input_layer")

x = base_model(inputs, training=False)
x = layers.GlobalAvgPool2D()(x)

x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)

x = tf.keras.layers.Dropout(0.5)(x)

outputs = layers.Dense(len(bird_classes), activation='softmax', dtype=tf.float32, kernel_regularizer=regularizers.l2(0.004))(x)

model1 = tf.keras.Model(inputs, outputs)

In [7]:
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 960)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               492032    
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dropout (Dropout)           (None, 512)               0     

In [8]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=3) 

In [9]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='Best_models/best_mobilenetv3large/', verbose=1, save_best_only=True)

history_eff_b0 = model1.fit(train_dataset,
                                        validation_data=validation_dataset,
                            steps_per_epoch=int(0.1*len(train_dataset)),
                            validation_steps=int(0.1*len(validation_dataset)),
                                        epochs=20,
                                        verbose=1,
                                        callbacks=[checkpointer]) 

Epoch 1/20


2024-03-23 10:52:18.152103: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-03-23 10:52:18.203267: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


 4/56 [=>............................] - ETA: 1s - loss: 5.8829 - accuracy: 0.0000e+00  

2024-03-23 10:52:19.045580: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5e5201def140 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-23 10:52:19.045601: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-03-23 10:52:19.053667: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-23 10:52:19.142607: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


54/56 [===========================>..] - ETA: 0s - loss: 5.7994 - accuracy: 0.0075
Epoch 1: val_loss improved from inf to 5.59365, saving model to Best_models/best_mobilenetv3large/
56/56 [==============================] - 13s 170ms/step - loss: 5.7946 - accuracy: 0.0078 - val_loss: 5.5936 - val_accuracy: 0.0000e+00
Epoch 2/20
55/56 [============================>.] - ETA: 0s - loss: 5.5747 - accuracy: 0.0125
Epoch 2: val_loss improved from 5.59365 to 5.30970, saving model to Best_models/best_mobilenetv3large/
56/56 [==============================] - 9s 155ms/step - loss: 5.5736 - accuracy: 0.0123 - val_loss: 5.3097 - val_accuracy: 0.0000e+00
Epoch 3/20
55/56 [============================>.] - ETA: 0s - loss: 5.3651 - accuracy: 0.0176
Epoch 3: val_loss improved from 5.30970 to 4.99112, saving model to Best_models/best_mobilenetv3large/
56/56 [==============================] - 9s 158ms/step - loss: 5.3638 - accuracy: 0.0173 - val_loss: 4.9911 - val_accuracy: 0.0000e+00
Epoch 4/20
55/56 [

In [10]:

for layer in base_model.layers[-100:]:
    layer.trainable = True


In [11]:
for layer in base_model.layers:
    print(layer.name, layer.trainable)

input_1 False
rescaling False
Conv False
Conv/BatchNorm False
tf.math.add False
re_lu False
tf.math.multiply False
multiply False
expanded_conv/depthwise False
expanded_conv/depthwise/BatchNorm False
re_lu_1 False
expanded_conv/project False
expanded_conv/project/BatchNorm False
expanded_conv/Add False
expanded_conv_1/expand False
expanded_conv_1/expand/BatchNorm False
re_lu_2 False
expanded_conv_1/depthwise/pad False
expanded_conv_1/depthwise False
expanded_conv_1/depthwise/BatchNorm False
re_lu_3 False
expanded_conv_1/project False
expanded_conv_1/project/BatchNorm False
expanded_conv_2/expand False
expanded_conv_2/expand/BatchNorm False
re_lu_4 False
expanded_conv_2/depthwise False
expanded_conv_2/depthwise/BatchNorm False
re_lu_5 False
expanded_conv_2/project False
expanded_conv_2/project/BatchNorm False
expanded_conv_2/Add False
expanded_conv_3/expand False
expanded_conv_3/expand/BatchNorm False
re_lu_6 False
expanded_conv_3/depthwise/pad False
expanded_conv_3/depthwise False
expa

In [12]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])
start_epoch = 20

model1.fit(train_dataset,
           steps_per_epoch=int(0.1*len(train_dataset)),
                            validation_steps=int(0.1*len(validation_dataset)),
                                        validation_data=validation_dataset,
                                        epochs=start_epoch+20,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer]) 

Epoch 21/40
55/56 [============================>.] - ETA: 0s - loss: 4.0933 - accuracy: 0.0716
Epoch 21: val_loss improved from 3.76138 to 3.26305, saving model to Best_models/best_mobilenetv3large/
56/56 [==============================] - 13s 177ms/step - loss: 4.0926 - accuracy: 0.0709 - val_loss: 3.2631 - val_accuracy: 0.0026
Epoch 22/40
55/56 [============================>.] - ETA: 0s - loss: 3.9462 - accuracy: 0.0767
Epoch 22: val_loss improved from 3.26305 to 3.02519, saving model to Best_models/best_mobilenetv3large/
56/56 [==============================] - 9s 163ms/step - loss: 3.9424 - accuracy: 0.0776 - val_loss: 3.0252 - val_accuracy: 0.2708
Epoch 23/40
54/56 [===========================>..] - ETA: 0s - loss: 3.7377 - accuracy: 0.1065
Epoch 23: val_loss improved from 3.02519 to 2.93483, saving model to Best_models/best_mobilenetv3large/
56/56 [==============================] - 9s 165ms/step - loss: 3.7347 - accuracy: 0.1083 - val_loss: 2.9348 - val_accuracy: 0.2721
Epoch 24/

In [13]:
len(base_model.layers)

263

In [14]:

for layer in base_model.layers[-150:]:
    layer.trainable = True


In [15]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])
start_epoch = 40

model1.fit(train_dataset,steps_per_epoch=int(0.1*len(train_dataset)),
                            validation_steps=int(0.1*len(validation_dataset)),
                                        
                                        validation_data=validation_dataset,
                                        
                                        epochs=start_epoch+20,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer]) 

Epoch 41/60
56/56 [==============================] - ETA: 0s - loss: 3.2646 - accuracy: 0.1507
Epoch 41: val_loss improved from 2.18635 to 1.96187, saving model to Best_models/best_mobilenetv3large/
56/56 [==============================] - 12s 165ms/step - loss: 3.2646 - accuracy: 0.1507 - val_loss: 1.9619 - val_accuracy: 0.3164
Epoch 42/60
54/56 [===========================>..] - ETA: 0s - loss: 3.1531 - accuracy: 0.1615
Epoch 42: val_loss improved from 1.96187 to 1.89828, saving model to Best_models/best_mobilenetv3large/
56/56 [==============================] - 9s 168ms/step - loss: 3.1608 - accuracy: 0.1590 - val_loss: 1.8983 - val_accuracy: 0.5195
Epoch 43/60
56/56 [==============================] - ETA: 0s - loss: 3.0728 - accuracy: 0.1819
Epoch 43: val_loss did not improve from 1.89828
56/56 [==============================] - 2s 28ms/step - loss: 3.0728 - accuracy: 0.1819 - val_loss: 2.2841 - val_accuracy: 0.2943
Epoch 44/60
55/56 [============================>.] - ETA: 0s - los

In [23]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(learning_rate=0.0001),
                           metrics=['accuracy'])
start_epoch = 160

model1.fit(train_dataset,
                                        validation_data=validation_dataset,
                                       
                                        epochs=start_epoch+20,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer]) 

Epoch 161/180
562/564 [============================>.] - ETA: 0s - loss: 0.2833 - accuracy: 0.9126
Epoch 161: val_loss did not improve from 0.33642
564/564 [==============================] - 17s 25ms/step - loss: 0.2835 - accuracy: 0.9125 - val_loss: 0.3566 - val_accuracy: 0.9079
Epoch 162/180
562/564 [============================>.] - ETA: 0s - loss: 0.2855 - accuracy: 0.9099
Epoch 162: val_loss did not improve from 0.33642
564/564 [==============================] - 14s 25ms/step - loss: 0.2853 - accuracy: 0.9099 - val_loss: 0.3433 - val_accuracy: 0.9121
Epoch 163/180
562/564 [============================>.] - ETA: 0s - loss: 0.2801 - accuracy: 0.9115
Epoch 163: val_loss did not improve from 0.33642
564/564 [==============================] - 14s 25ms/step - loss: 0.2799 - accuracy: 0.9115 - val_loss: 0.3471 - val_accuracy: 0.9259
Epoch 164/180
562/564 [============================>.] - ETA: 0s - loss: 0.2805 - accuracy: 0.9105
Epoch 164: val_loss did not improve from 0.33642
564/564 [

KeyboardInterrupt: 

In [26]:
loaded = tf.keras.models.load_model("Best_models/best_eff_b1.hdf5")


In [29]:
loaded.evaluate(test_dataset)

ValueError: in user code:

    File "/home/thefilthysalad/PycharmProjects/BIRD_CLASSIFICATION/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2042, in test_function  *
        return step_function(self, iterator)
    File "/home/thefilthysalad/PycharmProjects/BIRD_CLASSIFICATION/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2025, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/thefilthysalad/PycharmProjects/BIRD_CLASSIFICATION/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2013, in run_step  **
        outputs = model.test_step(data)
    File "/home/thefilthysalad/PycharmProjects/BIRD_CLASSIFICATION/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1895, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/thefilthysalad/PycharmProjects/BIRD_CLASSIFICATION/.venv/lib/python3.10/site-packages/keras/src/eng

In [15]:
train_data_path = "BIRDS1_split/train"
test_data_path = "BIRDS1_split/test"
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
bird_classes = sorted(os.listdir(train_data_path))

In [26]:
import tensorflow as tf
test_dataset_224 = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(224, 224),
    shuffle=False,
    seed=123,

)
test_dataset_299 = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(299, 299),
    shuffle=False,
    seed=123,

)
preprocess_input = tf.keras.applications.resnet.preprocess_input

test_dataset_224 = test_dataset_224.map(lambda x, y: (preprocess_input(x), y))
test_dataset_299 = test_dataset_299.map(lambda x, y: (preprocess_input(x), y))

Found 11545 files belonging to 167 classes.
Found 11545 files belonging to 167 classes.


In [27]:
y_labels_224 = []
y_labels_299 = []
for images, labels in test_dataset_224.unbatch():
    y_labels_224.append(labels.numpy().argmax())
for images, labels in test_dataset_299.unbatch():
    y_labels_299.append(labels.numpy().argmax())

In [18]:
def calculate_accuracy_top3(row):
    if row['True Label'] in row['top 3 cols']:
        return 1
    else:
        return 0

In [28]:
import pandas as pd

eff_b7 = tf.keras.models.load_model("Best_models/best_eff_b7_90.67")
preds_eff_b7 = pd.DataFrame(eff_b7.predict(test_dataset_224))
top3_labels = utils.add_top_three_row_values(preds_eff_b7, y_labels_224)
top3_labels['Accuracy_Top3'] = top3_labels.apply(calculate_accuracy_top3, axis=1)
overall_accuracy_top3 = top3_labels['Accuracy_Top3'].mean()
overall_accuracy_top3

361/361 [==============================] - 52s 136ms/step


0.9887397141619749

In [31]:
eff_v2L = tf.keras.models.load_model("Best_models/best_effnetv2L_90.57")

preds_eff_v2L = pd.DataFrame(eff_v2L.predict(test_dataset_224))

top3_labels_effv2L = utils.add_top_three_row_values(preds_eff_v2L, y_labels_224)

top3_labels_effv2L['Accuracy_Top3'] = top3_labels_effv2L.apply(calculate_accuracy_top3, axis=1)

overall_accuracy_top3_effv2L = top3_labels_effv2L['Accuracy_Top3'].mean()

overall_accuracy_top3_effv2L

361/361 [==============================] - 42s 107ms/step


0.9799913382416631

In [32]:
mobilenet_vsmall = tf.keras.models.load_model("Best_models/best_mobilenet_small_89.65")

preds_mobilenet_vsmall= pd.DataFrame(mobilenet_vsmall.predict(test_dataset_224))

top3_mobilenet_vsmall  = utils.add_top_three_row_values(preds_mobilenet_vsmall, y_labels_224)

top3_mobilenet_vsmall['Accuracy_Top3'] = top3_mobilenet_vsmall.apply(calculate_accuracy_top3, axis=1)

overall_accuracy_top3_mobilenet_vsmall= top3_mobilenet_vsmall['Accuracy_Top3'].mean()

overall_accuracy_top3_mobilenet_vsmall

361/361 [==============================] - 5s 12ms/step


0.9711563447379818

In [34]:
mobilenet_vlarge = tf.keras.models.load_model("Best_models/best_mobilenetv3large_90.67")

preds_mobilenet_vlarge = pd.DataFrame(mobilenet_vlarge.predict(test_dataset_224))

top3_mobilenet_vlarge  = utils.add_top_three_row_values(preds_mobilenet_vlarge, y_labels_224)

top3_mobilenet_vlarge['Accuracy_Top3'] = top3_mobilenet_vlarge.apply(calculate_accuracy_top3, axis=1)

overall_accuracy_top3_mobilenet_vlarge= top3_mobilenet_vlarge['Accuracy_Top3'].mean()

overall_accuracy_top3_mobilenet_vlarge

361/361 [==============================] - 7s 16ms/step


0.9778258986574274

In [5]:
import os 
labels = os.listdir("BIRDS1_split/train_165")
processed = []
for i in labels:
    i = i.lower()
    if ' ' in i:
        i.replace(' ','_')
    if '-' in i:
        i.replace('-','_')
    processed.append(i)
processed = sorted(processed)

In [28]:
loaded = tf.keras.models.load_model("Best_models/resnet_90.22")
loaded.evaluate(test_dataset_224)

361/361 [==============================] - 28s 75ms/step - loss: 0.3528 - accuracy: 0.9023


[0.352769136428833, 0.9022953510284424]

In [29]:
y_labels_224 = []
for images, labels in test_dataset_224.unbatch():
    y_labels_224.append(labels.numpy().argmax())


In [30]:
def calculate_accuracy_top3(row):
    if row['True Label'] in row['top 3 cols']:
        return 1
    else:
        return 0

In [32]:
import pandas as pd


preds_eff_b7 = pd.DataFrame(loaded.predict(test_dataset_224))
top3_labels = utils.add_top_three_row_values(preds_eff_b7, y_labels_224)
top3_labels['Accuracy_Top3'] = top3_labels.apply(calculate_accuracy_top3, axis=1)
overall_accuracy_top3 = top3_labels['Accuracy_Top3'].mean()
overall_accuracy_top3

361/361 [==============================] - 27s 70ms/step


0.9871805976613253